In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation as am

In [2]:
file = open("data_h_0.5.txt")

realisations = [] 
flag = True 
L = 63

for line in file.readlines(): 
    if line[0] == '#':
        flag = True 
        print(line)
    else: 
        if flag:  
            new_realisation = [line]
            realisations.append(new_realisation)
            flag = False
        else:
            new_realisation.append(line)

#Parameters={ Seed: 5, Hopping rate: 0.5, Realisations: 5, Chunk size: 5, Dimension: 2, (Max) Lattice size: 63, Graph Type: 0 }

#Version: #e4e6

# Running for L = 63

# Starting the 0 th realisation 

# Starting the 1 th realisation 

# Starting the 2 th realisation 

# Starting the 3 th realisation 

# Starting the 4 th realisation 

#okely dokely!


In [3]:
for (i, realisation) in enumerate(realisations):
    fig = plt.figure()
    ims = []
    xy = np.zeros((L, L))
    im = plt.imshow(xy, animated=True)
    plt.axis('off')
    ims.append([im])
    for line in realisation: 
        temp = np.fromstring(line[:-1], sep=',')
        t = temp[0]
        indices = np.unravel_index(temp[1:].astype('int'), (L, L))
        xy = np.zeros((L, L))
        xy[indices] = 1 
        im = plt.imshow(xy, animated=True)
        plt.axis('off')
        ims.append([im])
    ani = am.ArtistAnimation(fig, ims, interval=100, blit=True,
                                    repeat_delay=1000)
    mywriter = am.FFMpegWriter()
    ani.save("movie_{}.mp4".format(i), writer=mywriter)
    plt.close()